Welcome to ExoTiC-UVIS Stage 1! In this Stage, we will:

1. Load the data we downloaded in Stage 0.
2. Remove cosmic rays and hot pixels from the data through spatial and temporal filtering techniques.
3. Subtract the background signal using a myriad of different methods.
4. Track displacement of the trace across the detector over time.
5. Save our reduced data to an xarray file.

Make sure to run the ExoTiC-UVIS Stage 0 notebook before you run this one! This notebook relies on files that have been downloaded from Stage 0.

To get started, run the next cell, which imports the packages we need and creates a directory for ExoTiC-UVIS to operate from. Then follow the next markdown cell for instructions on how to execute Stage 1.

In [13]:
import os
from exotic_uvis import run_pipeline

config_directory = 'hustle_config'
if not os.path.exists(config_directory):
    os.makedirs(config_directory)

THE STAGE 1 .HUSTLE FILE

ExoTiC-UVIS operates using configuration files, designated with the .hustle file extension. Each .hustle file controls one Stage of ExoTiC-UVIS. .hustle files are more human-readable that raw code and they allow you to easily reproduce previous runs of ExoTiC-UVIS as well as rapidly sharing your reduction and analysis methods with your colleagues.

Now we need to make our Stage 1 .hustle file. The next cell contains a template stage_1.hustle file which you can modify to run Stage 1 of ExoTiC-UVIS. We're going to modify it so that it reduces the HUSTLE program observations of the hot Jupiter WASP-127b that we downloaded in Stage 0. To make it do this, make the following changes to the .hustle file template:

1. Set run_name to the string 'reduction_1'.
2. Set the verbose, show_plots, and save_plots variables to each be 0, 1, or 2. verbose controls how many printed statements the pipeline produces, letting you keep track of what the pipeline is doing and what step it is on. show_plots allows the pipeline to temporarily pause execution to show you an interactive plot. save_plots saves output .png or .gif files for plots and other graphics produced. 0 prints/shows/saves nothing, while 2 prints/shows/saves everything.
3. Set do_free_iter to True and free_sigma to 4.5. This tells ExoTiC-UVIS to iterate over each pixel's time series and replace outliers at the 4.5-sigma level until no more outliers are found.
4. Set do_uniform to True, fit to 'Gaussian', and hist_bins to 10000. This tells ExoTiC-UVIS to fit a single background value to each frame using the regions of the detector specified by the bounds variable.
5. Set do_location and do_0thtracking to True. Both of these methods refine the position of the direct image source location as well as the source's location in the spectroscopic images (known as the 0th order).

 As you modify each variable, take a moment to read the comment to its right. These comments tell you what each variable does and what else you can do with them.

In [14]:
hustle_stage_1_file = f"""
# ExoTiC-UVIS config file for launching Stage 1: Reduction

# Setup for Stage 1
toplevel_dir    'tutorial'                                  # Directory where your Stage 0 files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data.
run_name        None                                        # Str. This is the name of the current run. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      2                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      2                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data
skip_first_fm   False                                       # Bool. If True, ignores all first frames in each orbit.
skip_first_or   False                                       # Bool. If True, ignores all frames in the first orbit.

# Step 2: Reject cosmic rays with time iteration
# Step 2a: Fixed iteration parameters
do_fixed_iter   False                                       # Bool. Whether to use fixed iteration rejection to clean the timeseries.
fixed_sigmas    [10,10]                                     # lst of float. The sigma to reject outliers at in each iteration. The length of the list is the number of iterations.
replacement     None                                        # int or None. If int, replaces flagged outliers with the median of values within +/-replacement indices of the outlier. If None, uses the median of the whole timeseries instead.

# Step 2b: Free iteration parameters
do_free_iter    False                                       # Bool. Whether to use free iteration rejection to clean the timeseries.
free_sigma      3.5                                         # float. The sigma to reject outliers at in each iteration. Iterates over each pixel's timeseries until no outliers at this sigma level are found.

# Step 2c: Sigma clip parameters
do_sigma_clip   False                                       # Bool. Whether to use sigma clipping rejection to clean the timeseries.
# WIP!

# Step 3: Reject hot pixels with spatial detection
# Step 3a: Laplacian Edge Detection parameters
do_led          False                                       # Bool. Whether to use Laplacian Edge Detection rejection to clean the frames.
led_threshold   10                                          # Float. The threshold parameter at which to kick outliers in LED. The lower the number, the more values will be replaced.
led_factor      2                                           # Int. The subsampling factor. Minimum value 2. Higher values increase computation time but aren't expected to yield much improvement in rejection.
led_n           2                                           # Int. Number of times to do LED on each frame. Enter None to continue performing LED on each frame until no outliers are found.
fine_structure  False                                       # Bool. Whether to build a fine structure model, which can protect narrow bright features like traces from LED.
contrast_factor 5                                           # Float. If fine_structure is True, acts as the led_threshold for the fine structure step.

# Step 3b: Spatial smoothing parameters
do_smooth       False                                       # Bool. Whether to use spatial smoothing rejection to clean the frames.
# WIP

# Step 4: Background subtraction
# Step 4a: uniform value background subtraction
do_uniform      False                                       # Bool. Whether to subtract the background using one uniform value as the value for the entire frame.
fit             None                                        # Str. The value to extract from the histogram. Options are None (to extract the mode), 'Gaussian' (to fit the mode with a Gaussian), or 'median' (to take the median within hist_min < v < hist_max).
bounds          [[0,150,0,400],[440,590,0,400]]             # Lst of lst of float. The region from which the background values will be extracted. Each list consists of [x1,x2,y1,y2]. If None, simply uses the full frame.
hist_min        -20                                         # Float. Minimum value to consider for the background. Leave as None to use min(data).
hist_max        50                                          # Float. Maximum value to consider for the background. Leave as None to use max(data).
hist_bins       1000                                        # Int. Number of histogram bins for background subtraction.

# Step 4b: Column-by-column background subtraction
do_column       False                                       # Bool. Whether to subtract the background using a column-by-column method.
rows            [i for i in range(10)]                      # list of int. The indices defining the rows used as background.
mask_trace      True                                        # Bool. If True, ignores rows parameter and instead masks the traces and 0th order to build a background region.
dist_from_trace 100                                         # Int. If mask_trace is True, this is how many rows away a pixel must be from the trace to qualify as background.
col_sigma       3                                           # float. How aggressively to mask outliers in the background region.

# Step 4c: Pagul et al. background subtraction
do_Pagul        False                                       # Bool. Whether to subtract the background using the scaled Pagul et al. G280 sky image.
path_to_Pagul   './'                                        # Str. The absolute path to where the Pagul et al. G280 sky image is stored.
mask_parameter  0.001                                       # Float. How strong the trace masking should be. Smaller values mask more of the image.
smooth_fits     True                                        # Bool. If True, smooths the values of the Pagul et al. fit parameter in time. Helps prevent background "flickering".
smoothing_param 2.5                                         # Float. Sigma for smoothing the fit parameter. Smaller sigma means more smoothing.
median_columns  True                                        # Bool. If True, takes the median value of each column in the Pagul et al. sky image as the background. As the Pagul et al. 2023 image is undersampled, this helps to suppress fluctuations in the image.

# Step 5: Displacement estimation
# Step 5a: Refine target location
do_location     False                                       # Bool. Whether the location of the target in the direct image extracted from Stage 0 should be refined by fitting.

# Step 5b: Source center-of-mass tracking
do_0thtracking  False                                       # Bool. Whether to track frame displacements by centroiding the 0th order.           
location        [970, 170]                                  # lst of float. Initial guess for the location of the target star. You can use this to bypass location fitting in Stage 1.

# Step 5c: Background star tracking
do_bkg_stars    False                                       # Bool. Whether to track frame displacements by centroiding background stars.
bkg_stars_loc   [[0, 0], [0, 0]]                            # Lst of lst of float. Every list should indicate the estimated location of every background star.

# Step 6: Quality quicklook
do_quicklook    True                                        # Bool. Whether to perform this step.
gif_dir         'quicklook'                                 # str. Where to save the quicklook gif to, relative to the toplevel_dir.

# Step 7: Save outputs
do_save         True                                        # Bool. If True, saves the output xarray to be used in Stage 2.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_1_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_1_file)

In [15]:
hustle_stage_1_file = f"""
# ExoTiC-UVIS config file for launching Stage 1: Reduction

# Setup for Stage 1
toplevel_dir    'tutorial'                                  # Directory where your Stage 0 files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data.
run_name        'reduction_1'                               # Str. This is the name of the current run. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      0                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      1                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data
skip_first_fm   False                                       # Bool. If True, ignores all first frames in each orbit.
skip_first_or   False                                       # Bool. If True, ignores all frames in the first orbit.

# Step 2: Reject cosmic rays with time iteration
# Step 2a: Fixed iteration parameters
do_fixed_iter   False                                       # Bool. Whether to use fixed iteration rejection to clean the timeseries.
fixed_sigmas    [10,10]                                     # lst of float. The sigma to reject outliers at in each iteration. The length of the list is the number of iterations.
replacement     None                                        # int or None. If int, replaces flagged outliers with the median of values within +/-replacement indices of the outlier. If None, uses the median of the whole timeseries instead.

# Step 2b: Free iteration parameters
do_free_iter    True                                        # Bool. Whether to use free iteration rejection to clean the timeseries.
free_sigma      4.5                                         # float. The sigma to reject outliers at in each iteration. Iterates over each pixel's timeseries until no outliers at this sigma level are found.

# Step 2c: Sigma clip parameters
do_sigma_clip   False                                       # Bool. Whether to use sigma clipping rejection to clean the timeseries.
# WIP!

# Step 3: Reject hot pixels with spatial detection
# Step 3a: Laplacian Edge Detection parameters
do_led          False                                       # Bool. Whether to use Laplacian Edge Detection rejection to clean the frames.
led_threshold   5                                           # Float. The threshold parameter at which to kick outliers in LED. The lower the number, the more values will be replaced.
led_factor      2                                           # Int. The subsampling factor. Minimum value 2. Higher values increase computation time but aren't expected to yield much improvement in rejection.
led_n           2                                           # Int. Number of times to do LED on each frame. Enter None to continue performing LED on each frame until no outliers are found.
fine_structure  True                                        # Bool. Whether to build a fine structure model, which can protect narrow bright features like traces from LED.
contrast_factor 5                                           # Float. If fine_structure is True, acts as the led_threshold for the fine structure step.

# Step 3b: Spatial smoothing parameters
do_smooth       False                                       # Bool. Whether to use spatial smoothing rejection to clean the frames.
# WIP

# Step 4: Background subtraction
# Step 4a: uniform value background subtraction
do_uniform      True                                        # Bool. Whether to subtract the background using one uniform value as the value for the entire frame.
fit             'Gaussian'                                  # Str. The value to extract from the histogram. Options are None (to extract the mode), 'Gaussian' (to fit the mode with a Gaussian), or 'median' (to take the median within hist_min < v < hist_max).
bounds          [[0,150,0,400],[440,590,0,400]]             # Lst of lst of float. The region from which the background values will be extracted. Each list consists of [x1,x2,y1,y2]. If None, simply uses the full frame.
hist_min        -20                                         # Float. Minimum value to consider for the background. Leave as None to use min(data).
hist_max        50                                          # Float. Maximum value to consider for the background. Leave as None to use max(data).
hist_bins       10000                                       # Int. Number of histogram bins for background subtraction.

# Step 4b: Column-by-column background subtraction
do_column       False                                       # Bool. Whether to subtract the background using a column-by-column method.
rows            [i for i in range(10)]                      # list of int. The indices defining the rows used as background.
mask_trace      True                                        # Bool. If True, ignores rows parameter and instead masks the traces and 0th order to build a background region.
dist_from_trace 100                                         # Int. If mask_trace is True, this is how many rows away a pixel must be from the trace to qualify as background.
col_sigma       3                                           # float. How aggressively to mask outliers in the background region.

# Step 4c: Pagul et al. background subtraction
do_Pagul        False                                       # Bool. Whether to subtract the background using the scaled Pagul et al. G280 sky image.
path_to_Pagul   './'                                        # Str. The absolute path to where the Pagul et al. G280 sky image is stored.
mask_parameter  0.001                                       # Float. How strong the trace masking should be. Smaller values mask more of the image.
smooth_fits     True                                        # Bool. If True, smooths the values of the Pagul et al. fit parameter in time. Helps prevent background "flickering".
smoothing_param 2.5                                         # Float. Sigma for smoothing the fit parameter. Smaller sigma means more smoothing.
median_columns  True                                        # Bool. If True, takes the median value of each column in the Pagul et al. sky image as the background. As the Pagul et al. 2023 image is undersampled, this helps to suppress fluctuations in the image.

# Step 5: Displacement estimation
# Step 5a: Refine target location
do_location     True                                        # Bool. Whether the location of the target in the direct image extracted from Stage 0 should be refined by fitting.

# Step 5b: Source center-of-mass tracking
do_0thtracking  True                                        # Bool. Whether to track frame displacements by centroiding the 0th order.           
location        [970, 170]                                  # lst of float. Initial guess for the location of the target star. You can use this to bypass location fitting in Stage 1.

# Step 5c: Background star tracking
do_bkg_stars    False                                       # Bool. Whether to track frame displacements by centroiding background stars.
bkg_stars_loc   [[0, 0], [0, 0]]                            # Lst of lst of float. Every list should indicate the estimated location of every background star.

# Step 6: Quality quicklook
do_quicklook    True                                        # Bool. Whether to perform this step.
gif_dir         'quicklook'                                 # str. Where to save the quicklook gif to, relative to the toplevel_dir.

# Step 7: Save outputs
do_save         True                                        # Bool. If True, saves the output xarray to be used in Stage 2.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_1_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_1_file)

Now that our config file is ready, simply use the cell below to execute Stage 1 of the pipeline!

This Stage requires no user interaction unless show_plots is set to greater than 0, which prompts ExoTiC-UVIS to interrupt execution to show the user the plots being generated in an interactive format. If you have set show_plots to 0, then the pipeline will automatically finish running in about 3 minutes.

In [16]:
run_pipeline(config_files_dir=config_directory,
             stages=(1,))

Stage 0 .hustle located, using 'location' parameter supplied by Stage 0 fitting process.


Loading data... Progress:: 100%|██████████| 158/158 [00:07<00:00, 22.15it/s]
2024-11-15 14:52:12,705 - stpipe - WARNING - /Users/abbyboehm/opt/anaconda3/envs/hazel/lib/python3.11/site-packages/exotic_uvis/stage_1/temporal_outlier_rejection.py:185: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for i in tqdm(range(obs.dims['x']), desc = 'Removing cosmic rays and bad pixels... Progress:'):

Removing cosmic rays and bad pixels... Progress::   0%|          | 0/600 [00:00<?, ?it/s]2024-11-15 14:52:12,708 - stpipe - WARNING - /Users/abbyboehm/opt/anaconda3/envs/hazel/lib/python3.11/site-packages/exotic_uvis/stage_1/temporal_outlier_rejection.py:187: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with 

Free iterations complete. Total pixels corrected: 233112 out of 99540000


Removing background... Progress:: 100%|██████████| 79/79 [00:00<00:00, 125.15it/s]


Refining location of source in direct image with centroiding...


Tracking 0th order... Progress:: 100%|██████████| 79/79 [00:00<00:00, 2565.34it/s]


Tracked 0th order in 79 frames.


Parsing xarray for quicklookup... Progress:: 100%|██████████| 79/79 [00:00<00:00, 1249.16it/s]
2024-11-15 14:54:23,258 - stpipe - WARNING - MovieWriter ffmpeg unavailable; using Pillow instead.
2024-11-15 14:54:53,634 - stpipe - WARNING - MovieWriter ffmpeg unavailable; using Pillow instead.


Writing config file for Stage 1...
Config file written.


You made it! I hope there were no problems with execution. Now let's check out the outputs!

Inside the 'tutorial' directory should now be 'outputs/reduction_1/stage1'. This folder structure allows us to keep our runs separated so we can test many different reduction techniques on the same dataset. Inside that folder you will find:
- clean_obs.nc, which is an xarray file containing all of your reduced data
- stage_1_reduction_1.hustle, a copy of the config file you used to run this reduction
- plots/, a folder full of diagnostic plots that let you know how cleaning operated

You should also check the quicklook folder, which now has a Stage 1 folder in it. quicklookup.gif here is almost identical to the quicklookup.gif that we created in Stage 0, but now it shows the reduced data and its light curve. We also see quicklookupDQ.gif which shows the data quality (DQ) array over time. This gif is very handy in seeing where pixels are being flagged as bad. Ideally, a good cleaning should show a uniform distribution of bad pixels with no correlation to the + or - traces.

Depending on what you set verbose to, you may also see some print statements reporting on how things went, including how fixed iteration temporal outlier rejection caught 233,112 cosmic rays in our dataset. It's wise to check the plots/ directory and look at the CR_location.png file which would be created if you set save_plots >= 1. This plot shows all 233,112 cosmic rays and where they were found! A good sigma threshold should find cosmic rays distributed uniformly around the array. If your threshold is too low, you may find the traces are targeted too much. In this case, 4.5 sigma seems to be a reasonable threshold - cosmic rays are mostly uniform in their distribution, if slightly oversampling the +1 trace.

Let's see what happens when you change the threshold to be too low. Copy your config cell into the cell below and make the following change:

1. Set run_name to 'reduction_2'.
2. Set do_fixed_iter to True and fixed_sigmas to [1.5]. This will tell ExoTiC-UVIS to iterate over every pixel once and reject outliers at the 1.5-sigma level.
3. Set do_free_iter to False.

In [17]:
hustle_stage_1_file = f"""
# ExoTiC-UVIS config file for launching Stage 1: Reduction

# Setup for Stage 1
toplevel_dir    'tutorial'                                  # Directory where your Stage 0 files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data.
run_name        'reduction_2'                               # Str. This is the name of the current run. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      0                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      1                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data
skip_first_fm   False                                       # Bool. If True, ignores all first frames in each orbit.
skip_first_or   False                                       # Bool. If True, ignores all frames in the first orbit.

# Step 2: Reject cosmic rays with time iteration
# Step 2a: Fixed iteration parameters
do_fixed_iter   True                                        # Bool. Whether to use fixed iteration rejection to clean the timeseries.
fixed_sigmas    [1.5]                                       # lst of float. The sigma to reject outliers at in each iteration. The length of the list is the number of iterations.
replacement     5                                           # int or None. If int, replaces flagged outliers with the median of values within +/-replacement indices of the outlier. If None, uses the median of the whole timeseries instead.

# Step 2b: Free iteration parameters
do_free_iter    False                                       # Bool. Whether to use free iteration rejection to clean the timeseries.
free_sigma      3.5                                         # float. The sigma to reject outliers at in each iteration. Iterates over each pixel's timeseries until no outliers at this sigma level are found.

# Step 2c: Sigma clip parameters
do_sigma_clip   False                                       # Bool. Whether to use sigma clipping rejection to clean the timeseries.
# WIP!

# Step 3: Reject hot pixels with spatial detection
# Step 3a: Laplacian Edge Detection parameters
do_led          False                                       # Bool. Whether to use Laplacian Edge Detection rejection to clean the frames.
led_threshold   5                                           # Float. The threshold parameter at which to kick outliers in LED. The lower the number, the more values will be replaced.
led_factor      2                                           # Int. The subsampling factor. Minimum value 2. Higher values increase computation time but aren't expected to yield much improvement in rejection.
led_n           2                                           # Int. Number of times to do LED on each frame. Enter None to continue performing LED on each frame until no outliers are found.
fine_structure  True                                        # Bool. Whether to build a fine structure model, which can protect narrow bright features like traces from LED.
contrast_factor 5                                           # Float. If fine_structure is True, acts as the led_threshold for the fine structure step.

# Step 3b: Spatial smoothing parameters
do_smooth       False                                       # Bool. Whether to use spatial smoothing rejection to clean the frames.
# WIP

# Step 4: Background subtraction
# Step 4a: uniform value background subtraction
do_uniform      True                                        # Bool. Whether to subtract the background using one uniform value as the value for the entire frame.
fit             'Gaussian'                                  # Str. The value to extract from the histogram. Options are None (to extract the mode), 'Gaussian' (to fit the mode with a Gaussian), or 'median' (to take the median within hist_min < v < hist_max).
bounds          [[0,150,0,400],[440,590,0,400]]             # Lst of lst of float. The region from which the background values will be extracted. Each list consists of [x1,x2,y1,y2]. If None, simply uses the full frame.
hist_min        -20                                         # Float. Minimum value to consider for the background. Leave as None to use min(data).
hist_max        50                                          # Float. Maximum value to consider for the background. Leave as None to use max(data).
hist_bins       10000                                       # Int. Number of histogram bins for background subtraction.

# Step 4b: Column-by-column background subtraction
do_column       False                                       # Bool. Whether to subtract the background using a column-by-column method.
rows            [i for i in range(10)]                      # list of int. The indices defining the rows used as background.
mask_trace      True                                        # Bool. If True, ignores rows parameter and instead masks the traces and 0th order to build a background region.
dist_from_trace 100                                         # Int. If mask_trace is True, this is how many rows away a pixel must be from the trace to qualify as background.
col_sigma       3                                           # float. How aggressively to mask outliers in the background region.

# Step 4c: Pagul et al. background subtraction
do_Pagul        False                                       # Bool. Whether to subtract the background using the scaled Pagul et al. G280 sky image.
path_to_Pagul   './'                                        # Str. The absolute path to where the Pagul et al. G280 sky image is stored.
mask_parameter  0.001                                       # Float. How strong the trace masking should be. Smaller values mask more of the image.
smooth_fits     True                                        # Bool. If True, smooths the values of the Pagul et al. fit parameter in time. Helps prevent background "flickering".
smoothing_param 2.5                                         # Float. Sigma for smoothing the fit parameter. Smaller sigma means more smoothing.
median_columns  True                                        # Bool. If True, takes the median value of each column in the Pagul et al. sky image as the background. As the Pagul et al. 2023 image is undersampled, this helps to suppress fluctuations in the image.

# Step 5: Displacement estimation
# Step 5a: Refine target location
do_location     True                                        # Bool. Whether the location of the target in the direct image extracted from Stage 0 should be refined by fitting.

# Step 5b: Source center-of-mass tracking
do_0thtracking  True                                        # Bool. Whether to track frame displacements by centroiding the 0th order.           
location        [970, 170]                                  # lst of float. Initial guess for the location of the target star. You can use this to bypass location fitting in Stage 1.

# Step 5c: Background star tracking
do_bkg_stars    False                                       # Bool. Whether to track frame displacements by centroiding background stars.
bkg_stars_loc   [[0, 0], [0, 0]]                            # Lst of lst of float. Every list should indicate the estimated location of every background star.

# Step 6: Quality quicklook
do_quicklook    True                                        # Bool. Whether to perform this step.
gif_dir         'quicklook'                                 # str. Where to save the quicklook gif to, relative to the toplevel_dir.

# Step 7: Save outputs
do_save         True                                        # Bool. If True, saves the output xarray to be used in Stage 2.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_1_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_1_file)

In [18]:
run_pipeline(config_files_dir=config_directory,
             stages=(1,))

Stage 0 .hustle located, using 'location' parameter supplied by Stage 0 fitting process.


Correcting for 0th sigma... Progress:: 100%|██████████| 79/79 [00:18<00:00,  4.18it/s]
Iterating with fixed sigmas to remove CRs... Progess:: 1it [00:21, 21.12s/it]


Bad pixels removed on iteration 0 with sigma 1.50: 11791107
Fixed iterations complete. Total pixels corrected: 11791107 out of 99540000


Removing background... Progress:: 100%|██████████| 79/79 [00:00<00:00, 108.32it/s]


Refining location of source in direct image with centroiding...


Tracking 0th order... Progress:: 100%|██████████| 79/79 [00:00<00:00, 3321.94it/s]


Tracked 0th order in 79 frames.


Parsing xarray for quicklookup... Progress:: 100%|██████████| 79/79 [00:00<00:00, 1626.29it/s]
2024-11-15 14:56:19,390 - stpipe - WARNING - MovieWriter ffmpeg unavailable; using Pillow instead.
2024-11-15 14:56:54,978 - stpipe - WARNING - MovieWriter ffmpeg unavailable; using Pillow instead.


Writing config file for Stage 1...
Config file written.


Open 'outputs/reduction_2/stage1/plots/' and check out the CR_location.png file. If verbose >= 1, you should see that it reported a whopping 11,791,107 outliers. Practically every pixel was flagged as a cosmic ray at least once! That's definitely too much cleaning. The quicklookupDQ.gif file also shows the traces clearly appearing in some frames, meaning that large parts of real signal were flagged as outliers and replaced. That's not good! Make sure to make good use of these diagnostic plots to ensure your reduction is removing noise and not signal!

That's all for Stage 1! You can execute this Stage for any G280 time series observation you have downloaded as long as you run it through Stage 0's organization routine first. In the next tutorial notebook, we'll cover Stage 2, which calibrates the data using GRISMCONF and extracts the traces of our choosing.